# Project Mobile robotics
## filtering

what the fuck do we want?
Well a filter means having an estimated position at all time. And if we move we estimate where it is now, and merge this belief with what we see on the camera

And then we can always estimate where we are even if the camera is not on its best behavior


To have the position of robot we need x and y coordinates and orientation 
- sensing and performing a measurement update 
- moving and performing a motion update

In [2]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=d392bffcd6af1e3c5b48b013843495b364de7381ac638cc13d77c4232a10a846
  Stored in directory: /Users/Julie/Library/Caches/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy


In [3]:
import numpy as np
from filterpy.kalman import KalmanFilter

T_s = 1 # period en secondes selon laquelle on prend les mesures

# measurements i need from paul !! 
inital_x = 0
initial_y = 0 

measurement_noise_var = 1
model_noise_var = 1

In [ ]:
# chat gpt stuff
# Create a Kalman filter
kf = KalmanFilter(dim_x=4, dim_z=2)  # 4 states (x, y, vx, vy), 2 measurements (x, y)

# Define the initial state and covariance matrix
kf.x = np.array([initial_x, initial_y, initial_vx, initial_vy])
kf.P *= 1e3  # Initial covariance matrix

# Define the state transition matrix (constant velocity model)
kf.F = np.array([[1, 0, dt, 0],
                 [0, 1, 0, dt],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

# Define the measurement matrix (direct measurement of position)
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])

# Set the process noise covariance matrix
kf.Q = np.eye(4) * process_noise_variance

# Set the measurement noise covariance matrix
kf.R = np.eye(2) * measurement_noise_variance

# Kalman filter loop
for each_measurement in measurements:
    # Prediction step
    kf.predict()

    # Correction step
    kf.update(each_measurement)

    # Updated position estimate
    estimated_position = kf.x[:2]

    # Use the estimated_position for further processing or display

# Final estimated position
final_estimated_position = kf.x[:2]

# Kalman filter using module
- x - state: x, y, vx, vy (position and speed)
- z - measurements (z): x, y, o (position and orientation)
- P - covariance matrix: is updated to show the uncertainties of each states (describe how much each pair of state variables is correlated and how much uncertainty there is in the estimation of each individual state variable)
- F - state transition: dont know how to do it
- H- measurement: Identity as we measure x,y,o
- R- noise, uncertainty of measurements
- Q- noise, uncertainty of my model

In [ ]:
def create_kalman(initial_x, initial_y, initial_o, measurement_noise_var, model_noise_var):
    # create the filter
    dim_x = 3
    dim_z = 3
    kf = KalmanFilter(dim_x=dim_x, dim_z=dim_z)  # 4 states (x, y, o), 3 measurements (x, y, o)

    kf.x = np.array([inital_x, initial_y, initial_o]) # assuming we start when its stopped
    kf.P = np.eye(dim_x) # use identity for now

    # state transition model:
    kf.F = None # its a mess

    # measurements: 
    kf.H = np.eye(dim_z)

    # noise, can use variance of measurements ? NEED TO WORK MORE ON THIS
    kf.R = np.eye(dim_z) * measurement_noise_var
    kf.Q = np.eye(dim_x) * model_noise_var
    return kf

In [ ]:
def updating_filter(measurement):
    # Prediction step
    kf.predict()

    # Correction step
    kf.update(measurement)

    # Updated position estimate
    estimated_position = kf.x

    return estimated_position

In [ ]:
# code du prof
# Initialising the remaining constants
# units: length [mm], time [s]

A = np.array([[1, Ts], [0, 1]])
stripe_width = 50
Q = np.array([[qp, 0], [0, q_nu]]);
speed_conv_factor = 0.3375;
transition_thresh = 500

def kalman_filter(speed, pos_cam_prev, pos_cam, x_est_prev, P_est_prev,
                  HT=None, HNT=None, RT=None, RNT=None):
    """
    Estimates the current state using input sensor data and the previous state
    
    param speed: measured speed (Thymio units)
    param pos_cam_prev: previous value of measured position by camera
    param pos_cam: measured position by camera
    param x_est_prev: previous state a posteriori estimation
    param P_est_prev: previous state a posteriori covariance
    
    return pos_last_trans: updated if a transition has been detected
    return x_est: new a posteriori state estimation
    return P_est: new a posteriori state covariance
    """
    
    ## Prediciton through the a priori estimate
    # estimated mean of the state
    x_est_a_priori = np.dot(A, x_est_prev);
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if ((pos_cam_prev < transition_thresh)^(pos_cam < transition_thresh)) : #XOR (one or the other but not both)
        if (pos_cam>pos_cam_prev):
            stripe_width = 44
        else:
            stripe_width = 4
        # transition detected
        pos_last_trans = pos_last_trans + stripe_width;
        y = np.array([[pos_last_trans],[speed*speed_conv_factor]])
        H = np.array([[1, 0],[0, 1]])
        R = np.array([[rp, 0],[0, r_nu]])
    else:
        # no transition, use only the speed
        y = speed*speed_conv_factor;
        H = np.array([[0, 1]])
        R = r_nu;

    # innovation / measurement residual
    i = y - np.dot(H, x_est_a_priori);
    # measurement prediction covariance
    S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
             
    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
    
    
    # a posteriori estimate
    x_est = x_est_a_priori + np.dot(K,i);
    P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));
     
    return pos_last_trans, x_est, P_est